## Imports & display options

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from IPython.display import display

# Optional: nicer DataFrame display
pd.set_option('display.float_format', '{:,.4f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## Config & data loading

In [12]:
file_path = "activity-data.csv"
target_column = 'activity'
indexes_to_remove = []  # rows to drop (empty = keep all)

# Read CSV
df = pd.read_csv(file_path)

# Drop specified rows
df_cleaned = df.drop(index=indexes_to_remove)


## Target/feature split & basic cleaning

In [13]:
# Target
Y = df_cleaned[target_column]

# Features: drop target and identifier (if present)
X = df_cleaned.drop(columns=[target_column, 'Folder_Name'], errors='ignore')

# Fill NaNs with column means, then drop any columns still containing NaNs
X_filled = X.fillna(X.mean(numeric_only=True))
X_filled = X_filled.dropna(axis=1)

# Remove near-constant features
X_cleaned = X_filled.loc[:, X_filled.var() > 0.01]

X_cleaned.shape

(1013, 1391)

## Correlation pruning

In [14]:
corr = X_cleaned.corr(method='pearson').abs()
upper = np.triu(np.ones(corr.shape), k=1)

high_i, high_j = np.where((corr.values > 0.85) & (upper == 1))
drop_features = []
cols = corr.columns

for i, j in zip(high_i, high_j):
    fi, fj = cols[i], cols[j]
    if 'buried_volume' in fj:
        drop_features.append(fi)
    elif 'buried_volume' in fi:
        drop_features.append(fj)
    elif '_bol' in fj:
        drop_features.append(fi)
    elif '_bol' in fi:
        drop_features.append(fj)
    else:
        drop_features.append(fj)

drop_features = list(set(drop_features))
X_reduced = X_cleaned.drop(columns=drop_features, errors='ignore')

X_reduced.shape

(1013, 183)

## Scaling & PCA-derived top-20 features

In [15]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_reduced)

# PCA to retain 95% variance; compute weighted-squared-loadings importance
pca = PCA(n_components=0.95)
pca.fit(X_scaled)
loadings = pca.components_
explained = pca.explained_variance_ratio_
feature_importance = np.sum(explained.reshape(-1, 1) * loadings**2, axis=0)

# Sort and take top-20
sorted_idx = np.argsort(feature_importance)[::-1]
important_features = X_reduced.columns[sorted_idx]
top_20_features_PCA = important_features[:20]
X_PCA_top_20 = X_reduced[top_20_features_PCA]

list(top_20_features_PCA)

['6_Q_buried_volume_ave_scomformer',
 'Alpha_polarizability_bol',
 'MinEStateIndex',
 'time',
 '15_G4_bol',
 '4_G3_bol',
 'SlogP_VSA10',
 '1_G4_bol',
 'NumRotatableBonds',
 'SPS',
 'MinAbsEStateIndex',
 'R2_G3_bol',
 '6_Q_buried_volume_2_scomformer',
 'Distance_1_8_bol',
 '4_G4_bol',
 'PEOE_VSA6',
 'R2_G4_bol',
 '3_Q_buried_volume_min_scomformer',
 '15_G3_bol',
 '3_O_buried_volume_max_bol']

## Lasso- and RF-based feature selection

In [16]:
# LassoCV selection
lasso = LassoCV(cv=5, max_iter=10000, random_state=42)
lasso.fit(X_scaled, Y)
lasso_abs = np.abs(lasso.coef_)
top_20_features_lasso = X_reduced.columns[np.argsort(lasso_abs)[::-1][:20]]
X_LASSO_top_20 = X_reduced[top_20_features_lasso]

# Random Forest importance
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_reduced, Y)
rf_imp = rf.feature_importances_
top_20_features_rf = X_reduced.columns[np.argsort(rf_imp)[::-1][:20]]
X_RF_top_20 = X_reduced[top_20_features_rf]

# Combined (Lasso ∪ RF), deduplicated
X_combined = pd.concat([X_RF_top_20, X_LASSO_top_20], axis=1)
X_combined = X_combined.loc[:, ~X_combined.columns.duplicated()]

X_LASSO_top_20.shape, X_RF_top_20.shape, X_combined.shape

((1013, 20), (1013, 20), (1013, 32))

## Train/test split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_combined, Y, test_size=0.2, random_state=42)

# Refit scaler on the training slice for fair evaluation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled.shape, X_test_scaled.shape

((810, 32), (203, 32))

## Define regressors

In [18]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import ElasticNet, Lasso
from xgboost import XGBRegressor

models = {
    'ElasticNet': ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42),
    'RF': RandomForestRegressor(
        n_estimators=300, max_depth=6, min_samples_split=2, 
        min_samples_leaf=1, bootstrap=True, random_state=42
    ),
    'AdaBoost': AdaBoostRegressor(
        n_estimators=100, learning_rate=0.5, random_state=42
    ),
    'XGBoost': XGBRegressor(
        n_estimators=300, max_depth=4, learning_rate=0.1,
        subsample=0.8, colsample_bytree=0.9, random_state=42
    ),
    'LASSO': Lasso(alpha=0.1, random_state=42)
}

list(models.keys())

['ElasticNet', 'RF', 'AdaBoost', 'XGBoost', 'LASSO']

## Train, cross-validate, and evaluate

In [19]:
results = []

for model_name, model in models.items():
    # 5-fold CV on training set (neg MSE -> MSE)
    cv_scores = cross_val_score(
        model, X_train_scaled, y_train, cv=5,
        scoring='neg_mean_squared_error', n_jobs=-1
    )
    mean_cv_mse = -cv_scores.mean()

    # Fit
    model.fit(X_train_scaled, y_train)

    # Predict
    y_tr_pred = model.predict(X_train_scaled)
    y_te_pred = model.predict(X_test_scaled)

    # Metrics
    mse_train = mean_squared_error(y_train, y_tr_pred)
    mse_test  = mean_squared_error(y_test,  y_te_pred)
    r2_train  = r2_score(y_train, y_tr_pred)
    r2_test   = r2_score(y_test,  y_te_pred)
    mae_train = mean_absolute_error(y_train, y_tr_pred)
    mae_test  = mean_absolute_error(y_test,  y_te_pred)

    results.append([
        model_name, mse_train, mse_test, r2_train, r2_test, mae_train, mae_test, mean_cv_mse
    ])

results_df = pd.DataFrame(results, columns=[
    'Model', 'Train MSE', 'Test MSE', 'Train R²', 'Test R²', 'Train MAE', 'Test MAE', 'CV MSE'
])

display(results_df)

Model               Train MSE                Test MSE  Train R²  \
0  ElasticNet 26,785,981,736,965.0430 29,364,057,259,120.4141    0.3742   
1          RF  8,819,981,529,806.6250 12,964,443,881,520.8711    0.7940   
2    AdaBoost 15,887,191,726,458.7734 19,247,003,598,120.2969    0.6288   
3     XGBoost  1,693,007,216,640.0000  6,800,875,716,608.0000    0.9604   
4       LASSO 26,704,530,265,152.2617 29,108,349,469,874.9375    0.3761   

   Test R²      Train MAE       Test MAE                  CV MSE  
0   0.4093 3,553,278.0764 3,395,511.2444 29,101,558,263,905.8203  
1   0.7392 2,204,980.9650 2,487,597.1211 16,248,180,929,745.9570  
2   0.6128 3,151,802.9309 3,096,892.5702 21,706,061,619,785.8086  
3   0.8632   896,829.1250 1,856,756.3750 12,870,464,883,916.8008  
4   0.4144 3,570,232.9514 3,400,016.6077 29,565,476,628,941.8008

Params: 50, 1, 0.05, 0.4, 0.4 -> Test R²: 0.29717057943344116
Params: 50, 2, 0.1, 0.6, 0.8 -> Test R²: 0.5513488054275513
Params: 50, 3, 0.01, 0.9, 0.4 -> Test R²: 0.18397003412246704
Params: 50, 3, 0.1, 0.8, 0.6 -> Test R²: 0.7051209211349487
Params: 50, 4, 0.01, 0.9, 0.8 -> Test R²: 0.3085893392562866
Params: 50, 4, 0.2, 0.6, 1.0 -> Test R²: 0.771635115146637
Params: 50, 6, 0.1, 0.2, 0.6 -> Test R²: 0.6982885599136353
Params: 50, 6, 0.2, 0.6, 0.8 -> Test R²: 0.7751830816268921
Params: 50, 8, 0.05, 0.9, 0.2 -> Test R²: 0.40069711208343506
Params: 50, 8, 0.2, 0.4, 0.6 -> Test R²: 0.7641962766647339
Params: 100, 1, 0.2, 0.4, 0.8 -> Test R²: 0.5349382162094116
Params: 100, 2, 0.01, 1.0, 0.8 -> Test R²: 0.2948569059371948
Params: 100, 2, 0.1, 0.9, 1.0 -> Test R²: 0.6885897517204285
Params: 100, 3, 0.05, 0.4, 0.2 -> Test R²: 0.5262240171432495
Params: 100, 3, 0.1, 0.8, 1.0 -> Test R²: 0.7608947157859802
Params: 100, 4, 0.05, 0.6, 1.0 -> Test R²: 0.7206864953041077
Params: 100, 6, 0.01, 0.9